In [ ]:
import pandas as pd
import pickle
import requests
import json

# Verification of complete information

In [ ]:
definitionsE=["Group 1 (G1): Triplets of Generality or Equivalence. These relations usually correspond to Entailment. triplets: (ti,rel,hj) where ti is in Premise, hj is in Hypothesis and rel is the relation between ti an hj. e.g., (dog,is_a,animal) dog is in Premise, animal is in Hypothesis and relation is general between dog - animal.",
        "Group 2 (G2): Triplets of Contradictory. These often indicate Contradiction. triplets: (ti,rel,hj) where ti is in Premise, hj is in Hypothesis and rel is the relation between ti an hj. e.g., (dog,distinct_from,cat) dog is in Premise, cat is in Hypothesis and relation is distinct_from between dog - cat.",
        "Group 3 (G3): Triplets of Concretization, Specificity, or Contextuality. These typically correspond to Neutral. triplets: (hj, rel, ti), where ti is in Premise, hj is in Hypothesis, and rel is the relationship between ti and hj. For example, (dog, is_an, animal): animal is in Premise, dog is in Hypothesis, and the relationship is specific between animal - dog.",
        "Group 4 (G4): Relations not identified or categorized."]

gs=["G1","G2","G3","G4"]

gruposLLM=["ConteosG1","ConteosG2","ConteosG3","ConteosG4"]

# SKA prompt

In [ ]:
def callLLM_GS(model,f_,dfg,ind):
    prompt = '''
        You are an expert in Recognition of Textual Entailment over pairs of Premise and Hypothesis.
            Based on the information provided below, classify the relationship between the given Premise and Hypothesis as one of the following: "Entailment", "Neutral" or "Contradiction" and give an explanation. Respond only using the template:
            {
                "Answer": "",
                "Explanation":""
            }
            Do not modify the template.

            Background Information:
            
                Word Relationship Groups:
                    '''+str(definitionsE[f_])+'''

            Premise and Hypothesis to Classify:
                Premise: '''+dfg.at[ind,"Texto"]+'''
                Hypothesis: '''+ dfg.at[ind,"Hipotesis"]+'''
                
            Relations:
                '''+str(gs[f_])+''' : '''+str(dfg.at[ind,gruposLLM[f_]])+'''                     

            Use the information provided to classify the relationship and give an explanation.'''
    
    # print(prompt)
    # break
    data = {
        "prompt": prompt,
        "model": model,
        "format": "json",
        "stream": False,
        "options": {"temperature": 0},
    }
    try:
        response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=180)      
        json_data = json.loads(response.text)
        return True,json.dumps(json.loads(json_data["response"]), indent=2)
    except:
        print("Error")
        #print(response.text)
        #print(prompt)
    return False,str(response.text)

# Variables

In [ ]:
grupos=["G1","G2","G3","G4"]
m_n=14
ruta="../output2/relationships/"
modelos=["gemma2","gemma2:2b","llama3.1","llama3.2","phi3","phi3:medium"]
corpus=["Scitail","RTEGLUE","SNLI","SICK"] 

In [ ]:
conjunto_etiquetas=set()
for m in modelos:
    for c in corpus:
        print("Model",m,"Corpus",c)
        for g in range(len(grupos)):
            for i in range(m_n):
                new_data = {'Answer':[],'Explanation':[]}
                mt = pd.read_pickle(ruta+corpus.lower()+str(i+1)+".pickle")
                with open("processed/rit_"+str(i+1)+"_"+grupos[g]+".pickle", "rb") as f:
                    lista_respuestas = pickle.load(f)
                # Validar si faltan respuestas
                j=0
                for a in lista_respuestas:
                    try:
                        temp_dict=json.loads(a)
                        resp=temp_dict["Answer"]
                        exp=temp_dict["Explanation"]
                        new_data["Answer"].append(resp)
                        new_data["Explanation"].append(exp)
                    except:
                        print(j)
                        if g<4:
                            ex,dat=callLLM_GS(m,g,mt,j)
                        if ex==False:
                            if('\"Answer\": \"Neutral\"' in dat or '\\"Answer\\": \\"Neutral\\"' in dat):
                                new_data["Answer"].append("Neutral")
                            elif('\"Answer\": \"Entailment\"' in dat or '\\"Answer\\": \\"Entailment\\"' in dat):
                                new_data["Answer"].append("Entailment")
                            elif('\"Answer\": \"Contradiction\"' in dat or '\\"Answer\\": \\"Contradiction\\"' in dat):
                                new_data["Answer"].append("Contradiction")
                            else:
                                new_data["Answer"].append("")
                            new_data["Explanation"].append(dat)
                            print("a la segunda")
                        else:
                            temp_dict=json.loads(dat)
                            resp=temp_dict["Answer"]
                            exp=temp_dict["Explanation"]
                            new_data["Answer"].append(resp)
                            new_data["Explanation"].append(exp)
                            print("Hecho")
                    j+=1
                pd.DataFrame(new_data).to_pickle("../LLMs2/"+m+"/"+c+"/complete/rit_"+str(i+1)+"_"+grupos[g]+".pickle")
        

In [ ]:
m_n=1
ruta="../output2/relationships/"
modelos=["gemma2","gemma2:2b","llama3.1","llama3.2","phi3","phi3:medium"]
corpus=["DIAG"] 

In [ ]:
conjunto_etiquetas=set()
for m in modelos:
    for c in corpus:
        print("Model",m,"Corpus",c)
        for g in range(len(grupos)):
            for i in range(m_n):
                new_data = {'Answer':[],'Explanation':[]}
                mt = pd.read_pickle(ruta+corpus.lower()+str(i+1)+".pickle")
                with open("processed/rit_"+str(i+1)+"_"+grupos[g]+".pickle", "rb") as f:
                    lista_respuestas = pickle.load(f)
                # Validar si faltan respuestas
                j=0
                for a in lista_respuestas:
                    try:
                        temp_dict=json.loads(a)
                        resp=temp_dict["Answer"]
                        exp=temp_dict["Explanation"]
                        new_data["Answer"].append(resp)
                        new_data["Explanation"].append(exp)
                    except:
                        print(j)
                        if g<4:
                            ex,dat=callLLM_GS(m,g,mt,j)
                        if ex==False:
                            if('\"Answer\": \"Neutral\"' in dat or '\\"Answer\\": \\"Neutral\\"' in dat):
                                new_data["Answer"].append("Neutral")
                            elif('\"Answer\": \"Entailment\"' in dat or '\\"Answer\\": \\"Entailment\\"' in dat):
                                new_data["Answer"].append("Entailment")
                            elif('\"Answer\": \"Contradiction\"' in dat or '\\"Answer\\": \\"Contradiction\\"' in dat):
                                new_data["Answer"].append("Contradiction")
                            else:
                                new_data["Answer"].append("")
                            new_data["Explanation"].append(dat)
                            print("a la segunda")
                        else:
                            temp_dict=json.loads(dat)
                            resp=temp_dict["Answer"]
                            exp=temp_dict["Explanation"]
                            new_data["Answer"].append(resp)
                            new_data["Explanation"].append(exp)
                            print("Hecho")
                    j+=1
                pd.DataFrame(new_data).to_pickle("../LLMs2/"+m+"/"+c+"/complete/rit_"+str(i+1)+"_"+grupos[g]+".pickle")
        